In [1]:
import tensorflow as tf

In [3]:
a = tf.random.shuffle(tf.range(5))    #打乱顺序
a

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([2, 3, 4, 1, 0])>

In [5]:
tf.sort(a,direction='ASCENDING')     #升序排列

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>

In [6]:
tf.sort(a,direction='DESCENDING')     #降序排列

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 2, 1, 0])>

In [8]:
idx = tf.argsort(a)        #返回如何排序的
idx

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 0, 1, 2])>

In [9]:
tf.gather(a,idx)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>

#### 高维数据排序

In [11]:
a = tf.random.uniform([3,3],maxval=10,dtype=tf.int32)
a

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[2, 2, 1],
       [7, 3, 9],
       [9, 8, 7]])>

In [12]:
tf.sort(a)         #默认对最后一个维度进行排序

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 2],
       [3, 7, 9],
       [7, 8, 9]])>

In [13]:
tf.sort(a,direction='DESCENDING')

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[2, 2, 1],
       [9, 7, 3],
       [9, 8, 7]])>

In [14]:
tf.sort(a,axis=0)     #对给定维度进行排序

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[2, 2, 1],
       [7, 3, 7],
       [9, 8, 9]])>

#### Top_k 返回前几个最值及索引

In [15]:
a

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[2, 2, 1],
       [7, 3, 9],
       [9, 8, 7]])>

In [16]:
res = tf.math.top_k(a,2)    #返回两个最值

In [19]:
res.values     #返回最大的两个值

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[2, 2],
       [9, 7],
       [9, 8]])>

In [18]:
res.indices      #返回最大的两个值的索引

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[0, 1],
       [2, 0],
       [0, 1]])>

In [22]:
output = tf.random.normal([2,6])
output

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[-0.87764037,  0.8035606 , -1.1432153 ,  0.34809634,  0.89809996,
        -0.41597426],
       [-1.6270022 , -1.240233  ,  0.31657612, -0.5907291 ,  0.06888622,
        -0.8796589 ]], dtype=float32)>

In [27]:
output = tf.math.softmax(output)     #使六个数据概率总和为1
output

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.14815383, 0.18879174, 0.14625038, 0.1693017 , 0.19445011,
        0.15305232],
       [0.14746293, 0.15101531, 0.19938007, 0.16162139, 0.18456003,
        0.15596028]], dtype=float32)>

In [28]:
tf.reduce_sum(output[0])

<tf.Tensor: shape=(), dtype=float32, numpy=1.0000001>